In [ ]:
import warnings
%cd ~/projects/cardiovascular-risk/
warnings.filterwarnings('ignore')
%load_ext autoreload
%load_ext rpy2.ipython

%load_ext ipycache
%autoreload 2

# these are the defaults we want to set:
default_units = 'in' # inch, to make it more easily comparable to matpplotlib
default_res = 200 # dpi, same as default in matplotlib
default_width = 15
default_height = 8
# try monkey-patching a function in rpy2, so we effectively get these
# default settings for the width, height, and units arguments of the %R magic command
import rpy2
old_setup_graphics = rpy2.ipython.rmagic.RMagics.setup_graphics

def new_setup_graphics(self, args):
    if getattr(args, 'units') is not None:
        if args.units != default_units: # a different units argument was passed, do not apply defaults
            return old_setup_graphics(self, args)
    args.units = default_units
    if getattr(args, 'res') is None:
        args.res = default_res
    if getattr(args, 'width') is None:
        args.width = default_width
    if getattr(args, 'height') is None:
        args.height = default_height        
    return old_setup_graphics(self, args)

rpy2.ipython.rmagic.RMagics.setup_graphics = new_setup_graphics

## Loading data

In [ ]:
%%R

rm(list = ls())
setwd('~/projects/cardiovascular-risk/')
Heart <- read.csv("../cardiovascular-risk-data/data/UNOS.csv")


## Function definition

In [ ]:
%%R


###########
#functions#
###########
par(mfrow = c(1, 1))

barsum <- function(x, title) {
  barplot(table(x), col = "royalblue3", main = title, res=2000)
}

histsum <- function(x, title) {
  N <- length(x)
  x <- na.omit(x)
  m <- mean(x)
  std <- sqrt(var(x))
  lh <- quantile(x, probs = 0.25)
  uh <- quantile(x, probs = 0.75)
  step <- 1.5 * (uh - lh)
  outlier <- x < lh - step | x > uh + step
  #hist(x[!outlier], col = "royalblue3", ylab = "probability", probability = TRUE)
  #h = sd(x)*(4/3/length(x))^(1/5)
  #lines(density(x[!outlier],bw = h), col = "red")
  #curve(dnorm(x,mean=m,sd=std),col ="green",add = TRUE)
  hist(x, col = "royalblue3", ylab = "probability", probability = TRUE, main = title)
  h <- sd(x) * (4 / 3 / length(x))^(1 / 5)
  lines(density(x, bw = h), col = "red")
  curve(dnorm(x, mean = m, sd = std), col = "green", add = TRUE)
  rug(x)
}

histsumoutlier <- function(x) {
  N <- length(x)
  x <- na.omit(x)
  m <- mean(x)
  std <- sqrt(var(x))
  lh <- quantile(x, probs = 0.25)
  uh <- quantile(x, probs = 0.75)
  step <- 1.5 * (uh - lh)
  outlier <- x < lh - step | x > uh + step
  hist(x[!outlier], col = "royalblue3", ylab = "probability", probability = TRUE)
  h <- sd(x) * (4 / 3 / length(x))^(1 / 5)
  lines(density(x[!outlier], bw = h), col = "red")
  curve(dnorm(x, mean = m, sd = std), col = "green", add = TRUE)
  rug(x)
  print(N - length(x))
}

## Histograms

In [ ]:
%%R
###########
#diagnosis#
###########
barsum(Heart$thoracic_dgn, 'thoracic_dgn-not neeeded')
#not needed, only look at diag

In [ ]:
%%R

########
#gender#
########
barsum(Heart$gender, 'gender')

In [ ]:
%%R

############
#bloodgroup#
############
barsum(Heart$abo, 'blood group')
# Z="unknown"

In [ ]:
%%R

#weight recipient#
histsum(Heart$wgt_kg_tcr, 'weight recipient')
logwgt <- log10(Heart$wgt_kg_tcr)
histsum(logwgt, 'log weight recipient')
#3969 missing values <15%
#logtransform, multimodal
#group adults, group babies

In [ ]:
%%R

#height recipient
histsum(Heart$hgt_cm_tcr, 'height recipient')
loghgt <- log10(Heart$hgt_cm_tcr)
histsum(loghgt, 'log height recipient')
#4933 missing values <15%
#logtransform

In [ ]:
%%R

#intra-aortic blood pump
barsum(Heart$iabp_tcr, 'intra-aortic blood pump')

In [ ]:
%%R

#inotropica
barsum(Heart$inotropes_tcr, 'inotropica')

In [ ]:
%%R

#functional status at listing recipient
barsum(Heart$func_stat_tcr, 'functional status at listing recipient')
#1-3 = DAL assistance needed? 996= not applicable, 998 = unknown
#'20##': percentage function in adults
#'40##': percentage function in children
#2640 missing

In [ ]:
%%R

#functional status at transplant recipient
barsum(Heart$func_stat_trr, 'functional status at transplant recipient')
#1-3 = DAL assistance needed? 996= not applicable, 998 = unknown
#'20##': percentage function in adults
#'40##': percentage function in children
#62472 missing (vs TCR?)

In [ ]:
%%R

#diabetes
barsum(Heart$diab, 'diabetes')
#usually subdivision in type 1 and type 2 diabetes
#possible subdivison: 1 = no diabetes, 2 = diabetes I, 3 = diabetes II, 998 = unknown, 2/4 ???
#4 = type other, 5 = type unknown 998 = unknown
#45888 missing

In [ ]:
%%R

#cerebrovascular disease
barsum(Heart$cereb_vasc, 'cerebrovascular disease')
#possible subdivision N = no CVD, Y = CVD, U = unknown, 46483 missing?
#46483 missing

In [ ]:
%%R

#most recent creatinine (kidney function)
histsum(Heart$most_rcnt_creat, 'most recent creatinine (kidney function)')
logcreat <- log10(Heart$most_rcnt_creat)
histsum(logcreat, 'log most recent creatinine (kidney function)')
#72569 missing > 15%
#logtransform

In [ ]:
%%R

#serum albumine
histsum(Heart$tot_serum_album, 'serum albumine')
#99944 missing > 15%

In [ ]:
%%R

#sudden death
barsum(Heart$sud_death, 'sudden death')
#possible subdivision N = no, Y = yes, U = unknown, 92957 missing data
#92957 missing

In [ ]:
%%R

#defibrillator in recipient
barsum(Heart$impl_defibril, 'defibrillator in recipient')
#possible subdivision N = no, Y = yes, U = unknown, 45923 missing data
#45923 missing

In [ ]:
%%R

#systolic pulmonary pressure recipient (mmHg)
histsum(Heart$hemo_sys_tcr, 'systolic pulmonary pressure recipient (mmHg)')
#59366 missing > 15%

In [ ]:
%%R

#diastolic pulmonary pressure recipient (mmHg)
histsum(Heart$hemo_pa_dia_tcr, 'diastolic pulmonary pressure recipient (mmHg)')
#59573 missing > 15%

In [ ]:
%%R

#mean pulmonary pressure recipient (mmHg)
histsum(Heart$hemo_pa_mn_tcr, 'mean pulmonary pressure recipient (mmHg)')
#61370 missing >15%

In [ ]:
%%R

#mean pulmonary wedge pressure recipient (mmHg)
histsum(Heart$hemo_pcw_tcr, 'mean pulmonary wedge pressure recipient (mmHg)')
#64146 missing >15%

In [ ]:
%%R

#cardiac output recipient (L/min)
histsum(Heart$hemo_co_tcr, 'cardiac output recipient (L/min)')
#64872 missing >15%

In [ ]:
%%R

# cigarette use recipient
barsum(Heart$cig_use, ' cigarette use recipient')
#possible N = no, Y = yes
#89926 missing

In [ ]:
%%R

#prior cardiac surgery recipient
barsum(Heart$prior_card_surg_tcr, 'prior cardiac surgery recipient')
#possible subdivision N = no, Y = yes, U = unknown,
#6748 missing data

In [ ]:
%%R

#prior cardiac surgery recipient type
barsum(Heart$prior_card_surg_type_tcr, 'prior cardiac surgery recipient type')
# 130560 missing, see CARDSURG

In [ ]:
%%R

#ethnicity
barsum(Heart$ethcat, 'ethnicity')
# possible subdivision: 1 = white, 2 = black
# 4= hispanic, 5=asian,6=alaska native,
#7= native hawaiian,9= multiracial, 998 unknown
#447 unknown

In [ ]:
%%R

#recipient on ventilation
barsum(Heart$ventilator_tcr, 'recipient on ventilation')

In [ ]:
%%R

#lvad.ever
barsum(Heart$lvad.ever, 'lvad.ever')

In [ ]:
%%R


#rvad.ever, right ventricle assist device
barsum(Heart$rvad.ever, 'rvad.ever, right ventricle assist device')

In [ ]:
%%R


#tah.ever, total artificial heart
barsum(Heart$tah.ever, 'tah.ever, total artificial heart')

In [ ]:
%%R


#medical condition recipient
barsum(Heart$med_cond_trr, 'medical condition recipient')
#62159 missing
#1 = ICU, 2 = hospital not ICU, 3 = not hospitalized

In [ ]:
%%R

#ecmo at transplant
barsum(Heart$pstatus, 'ecmo at transplant')
#61862 missing

In [ ]:
%%R

#creatinine at transplant
histsum(Heart$creat_trr, 'creatinine at transplant')
logcreat_trr <- log10(Heart$creat_trr)
histsum(lncreat_trr, 'log creatinine at transplant')
#logtransform
#76567 missing >15%

In [ ]:
%%R

#dialysis after listing
barsum(Heart$dial_after_list, 'dialysis after listing')
#possible subdivision N = no, Y = yes, U =unknown, 76083 missing data
#76083 missing

In [ ]:
%%R

#bilirubine recipient
histsum(Heart$tbili, 'bilirubine recipient')
logbili <- log10(Heart$tbili)
histsum(logbili, 'log bilirubine recipient')
#80015 missing >15%

In [ ]:
%%R

#blood transfusions since listing
barsum(Heart$transfusions, 'blood transfusions since listing')
#possible subdivision N = no, Y = yes, U =unknown, 70535 missing data
#70535 missing

In [ ]:
%%R

#hepatitis b virus core
barsum(Heart$hbv_core, 'hepatitis b virus core')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 72576 missing data
#72576 missing

In [ ]:
%%R

#hepatitis b virus surface antigen
barsum(Heart$hbv_sur_antigen, 'hepatitis b virus surface antigen')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 72576 missing data
#missing 72576

In [ ]:
%%R

#cmv status
barsum(Heart$cmv_status, 'cmv status')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 71784 missing data
#71784 missing

In [ ]:
%%R

#cmv igg
barsum(Heart$cmv_igg, 'cmv igg')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 83452 missing data
#83452 missing

In [ ]:
%%R

#cmv igm
barsum(Heart$cmv_igm, 'cmv igm')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 83454 missing data
#83454 missing

In [ ]:
%%R

#hiv status
barsum(Heart$hiv_serostatus, 'hiv status')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 71795 missing data
#71795 missing

In [ ]:
%%R

#hcv_serostatus
barsum(Heart$hcv_serostatus, 'hcv_serostatus')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 72871 missing data
#72871 missing

In [ ]:
%%R

#ebv_serostatus
barsum(Heart$ebv_serostatus, 'ebv_serostatus')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, 92569 missing data
#92569 missing

In [ ]:
%%R

#survival of graft
barsum(Heart$gstatus, 'survival of graft')
#possible subdivision 0 = death or reimplantation, 1 = survival
#61949 missing

In [ ]:
%%R

#time of graft survival
histsum(Heart$gtime, 'time of graft survival')
loggtime <- log10(Heart$gtime + 1)
histsum(loggtime, 'log time of graft survival')
#61949 missing, logtransform

In [ ]:
%%R

#survival of patient
barsum(Heart$pstatus, 'survival of patient')
#possible subdivision 0 = death, 1 = survival
#61949 missing

In [ ]:
%%R

#time of patient survival
histsum(Heart$ptime, 'time of patient survival')
logptime <- log10(Heart$ptime + 1)
histsum(logptime, 'log time of patient survival')
#61949 missing, logtransform

In [ ]:
%%R

#status patient?
barsum(Heart$px_stat, 'status patient?')
#possible subdivision D - dead, A - alive, R - transplanted, L?, 61945 missing data
#61945 missing

In [ ]:
%%R

#previous transplantation
barsum(Heart$prev_tx, 'previous transplantation')
#61862 missing data

In [ ]:
%%R

#previous transplantation
barsum(Heart$prev_tx_any, 'previous transplantation')
#61862 missing data

In [ ]:
%%R

#history of cocaine use donor
barsum(Heart$hist_cocaine_don, 'history of cocaine use donor')
#possible subdivision N = no, Y = yes, U =unknown, 61964 missing data
#61964 missing

In [ ]:
%%R

#age donor
histsum(Heart$age_don, 'age donor')
logagedon <- log10(Heart$age_don + 1)
histsum(logagedon, 'log age donor')
#61870 missing, logtransform

In [ ]:
%%R

#ethnicity donor
barsum(Heart$ethcat_don, 'ethnicity donor')
# possible subdivision: 1 = white, 2 = black
#4 = hispanic, 5 = asian, 6 = native alaska,
#7 = native hawaiiunclear, 9 = multiracial, 998 unknown?
#61862 missing

In [ ]:
%%R

#hepatitis b virus core donor
barsum(Heart$hbv_core_don, 'hepatitis b virus core donor')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, C/I unclear, 76707 missing data
#76707 missing
#codes unclear

In [ ]:
%%R

#hepatitis b virus surface antigen donor
barsum(Heart$hbv_sur_antigen_don, 'hepatitis b virus surface antigen donor')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, C/I unclear, 62024 missing data
#62024 missing

In [ ]:
%%R

#bloodgroup#
barsum(Heart$abo_don, 'bloodgroup')

In [ ]:
%%R

#heavy alcohol use donor
barsum(Heart$alcohol_heavy_don, 'heavy alcohol use donor')
#possible subdivision N = no, Y = yes, U =unknown, 106369 missing data
#106369 missing

In [ ]:
%%R

#gender#
barsum(Heart$gender_don, 'gender')
#missing data 61863

In [ ]:
%%R

#hcv_serostatus donor
barsum(Heart$hep_c_anti_don, 'hcv_serostatus donor')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, I = not clear 76715 missing data
#76715 missing

In [ ]:
%%R

# non heart beating donor
barsum(Heart$non_hrt_don, 'non heart beating donor')
#76662 missing

In [ ]:
%%R

#infection in blood source donor
barsum(Heart$blood_inf_don, 'infection in blood source donor')
#61905 missing

In [ ]:
%%R

#blood urea nitrogen at transplantation in donor
histsum(Heart$bun_don, 'blood urea nitrogen at transplantation in donor')
logbundon <- log10(Heart$bun_don)
histsum(loggbundon, 'log blood urea nitrogen at transplantation in donor')
#76931 missing > 15%, logtransform

In [ ]:
%%R

#creatinine at transplantation in donor
histsum(Heart$creat_don, 'creatinine at transplantation in donor')
logcreat_don <- log10(Heart$creat_don)
histsum(logcreat_don, 'log creatinine at transplantation in donor')
#76921 missing > 15%, logtransform

In [ ]:
%%R

#other infection at transplantation in donor
barsum(Heart$other_inf_don, 'other infection at transplantation in donor')
#119490 missing

In [ ]:
%%R

#pulmonary infection at transplantation in donor
barsum(Heart$pulm_inf_don, 'pulmonary infection at transplantation in donor')
#61905 missing

In [ ]:
%%R

#AST at transplantation in donor
histsum(Heart$sgot_don, 'AST at transplantation in donor')
logsgot_don <- log10(Heart$sgot_don)
histsum(logsgot_don, 'log AST at transplantation in donor')
#77404 missing >15%, logtransform

In [ ]:
%%R

#ALT at transplantation in donor
histsum(Heart$sgpt_don, 'ALT at transplantation in donor')
logsgpt_don <- log10(Heart$sgpt_don)
histsum(logsgpt_don, 'log ALT at transplantation in donor')
#77872 missing >15%, logtransform

In [ ]:
%%R

#bilirubine at transplantation in donor
histsum(Heart$tbili_don, 'bilirubine at transplantation in donor')
logtbili_don <- log10(Heart$tbili_don + 1)
histsum(logtbili_don, 'log bilirubine at transplantation in donor')
#77621 missing >15%, logtransform

In [ ]:
%%R

#urinal infection at transplantation in donor
barsum(Heart$urine_inf_don, 'urinal infection at transplantation in donor')
#61905 missing

In [ ]:
%%R

#vasodilatators at transplantation in donor
barsum(Heart$vasodil_don, 'vasodilatators at transplantation in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76767 missing

In [ ]:
%%R

#RPR-VDRL SEROLOGY at transplantation in donor
barsum(Heart$vdrl_don, 'RPR-VDRL SEROLOGY at transplantation in donor')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, death_mech_don
#62067 missing

In [ ]:
%%R

#clinical infection at transplantation in donor
barsum(Heart$clin_infect_don, 'clinical infection at transplantation in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76768 missing

In [ ]:
%%R

#cocaine use in last 6 months in donor
barsum(Heart$contin_cocaine_don, 'cocaine use in last 6 months in donor')
#possible subdivision N = no, Y = yes, U =unknown
#145405 missing

In [ ]:
%%R

#other drug use in last 6 months in donor
barsum(Heart$contin_oth_drug_don, 'other drug use in last 6 months in donor')
#possible subdivision N = no, Y = yes, U =unknown
#116064 missing

In [ ]:
%%R

#extracranial cancer in donor at transplantation
barsum(Heart$extracranial_cancer_don, 'extracranial cancer in donor at transplantation')
#possible subdivision N = no, Y = yes, U =unknown
#76876 missing

In [ ]:
%%R

#history of alcohol in donor
barsum(Heart$hist_alcohol_old_don, 'history of alcohol in donor')
#possible subdivision N = no, Y = yes, U =unknown
#123356 missing

In [ ]:
%%R

#site of cancer in donor
barsum(Heart$cancer_site_don, 'site of cancer in donor')
#possible subdivision 1 = no cancer, cancer_site_don
#missing 76846

In [ ]:
%%R

#history of cigarettes abuse in donor
barsum(Heart$hist_cig_don, 'history of cigarettes abuse in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76731 missing

In [ ]:
%%R

#history of hypertension in donor
barsum(Heart$hist_hypertens_don, 'history of hypertension in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76734 missing

In [ ]:
%%R

#history of IV drugs in donor
barsum(Heart$hist_iv_drug_old_don, 'history of IV drugs in donor')
#possible subdivision N = no, Y = yes, U =unknown
#123371 missing

In [ ]:
%%R

#intracranial cancer in donor at transplantation
barsum(Heart$intracranial_cancer_don, 'intracranial cancer in donor at transplantation')
#possible subdivision N = no, Y = yes, U =unknown
#76876 missing

In [ ]:
%%R

#history of cancer in donor
barsum(Heart$hist_cancer_don, 'history of cancer in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76846 missing

In [ ]:
%%R

#insulin dependence in donor
barsum(Heart$donor.insulin, 'insulin dependence in donor')
#possible subdivision N = no, Y = yes
#76742 missing

In [ ]:
%%R

#diabetes in donor
barsum(Heart$diabetes_don, 'diabetes in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76742 missing

In [ ]:
%%R

#history of other drug abuse in donor
barsum(Heart$hist_oth_drug_don, 'history of other drug abuse in donor')
#possible subdivision N = no, Y = yes, U =unknown
#76766 missing

In [ ]:
%%R

#skin cancer in donor at transplantation
barsum(Heart$skin_cancer_don, 'skin cancer in donor at transplantation')
#possible subdivision N = no, Y = yes, U =unknown
#76876 missing

In [ ]:
%%R

#cmv infection in donor
barsum(Heart$cmv_don, 'cmv infection in donor')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, C/I unclear
#62052 missing

In [ ]:
%%R

#donor cause of death
barsum(Heart$cod_cad_don, 'donor cause of death')
#62013 missing, codes missing

In [ ]:
%%R

#death_mech_don
barsum(Heart$death_mech_don, 'death_mech_don')
#colv_eject_meth
#62022 missing

In [ ]:
%%R

#calculated donor height donor
histsum(Heart$hgt_cm_don_calc, 'calculated donor height donor')
loghgtdon <- log10(Heart$hgt_cm_don_calc)
histsum(loghgtdon, 'log calculated donor height donor')
#71170 missing >15%, logtransform

In [ ]:
%%R

#calculated donor weight donor
histsum(Heart$wgt_kg_don_calc, 'calculated donor weight donor')
logwgtdon <- log10(Heart$wgt_kg_don_calc)
histsum(logwgtdon, 'log calculated donor weight donor')
#65656 missing >15%, logtransform

In [ ]:
%%R

#calculated BMI donor
histsum(Heart$bmi_don_calc, 'calculated BMI donor')
#71235 missing > 15%

In [ ]:
%%R

#multiorgan transplant
barsum(Heart$multiorg, 'multiorgan transplant')

In [ ]:
%%R

#transplant match in bloodgroup
barsum(Heart$abo_mat, 'transplant match in bloodgroup')
#possible subdivision 1 = match, 2 = no match, 3 =unknown; codes needed
#61867 missing

In [ ]:
%%R

#age recipient
histsum(Heart$age, 'age recipient')
logage <- log10(Heart$age + 1)
histsum(logage, 'log age recipient')
#61871 missing >15%, logtransform

In [ ]:
%%R

#thoracic diagnosis recipient
barsum(Heart$diag, 'thoracic diagnosis recipient')
#61864 missing

In [ ]:
%%R

#dialysis prior to transplantation
barsum(Heart$dial_prior_tx, 'dialysis prior to transplantation')
#possible subdivision N = no, Y = yes, U =unknown
#80217 missing

In [ ]:
%%R

#ischemic time of donor in hours
histsum(Heart$ischtime, 'ischemic time of donor in hours')
logisch <- log10(Heart$ischtime + 1)
histsum(logisch, 'log ischemic time of donor in hours')
#2 groups
#no missing, logtransform

In [ ]:
%%R

#patient on life support
barsum(Heart$life_sup_trr, 'patient on life support')
#possible subdivision N = no, Y = yes
#62150 missing

In [ ]:
%%R

#prior cardiac surgery between listing and transplantation
barsum(Heart$prior_card_surg_trr, 'prior cardiac surgery between listing and transplantation')
#possible subdivision N = no, Y = yes
#66791 missing

In [ ]:
%%R

#any previous cancers
barsum(Heart$malig, 'any previous cancers')
#possible subdivision N = no, Y = yes,U = unknown
#61876 missing

In [ ]:
%%R

#calculated height recipient in cm
histsum(Heart$hgt_cm_calc, 'calculated height recipient in cm')
loghgtcalc <- log10(Heart$hgt_cm_calc)
histsum(loghgtcalc, 'log calculated height recipient in cm')
#62737 missing >15%, logtransform
#difference with hgt_cm_tcr?

In [ ]:
%%R

#calculated weight recipient in kg
histsum(Heart$wgt_kg_calc, 'calculated weight recipient in kg')
logwgtcalc <- log10(Heart$wgt_kg_calc)
histsum(logwgtcalc, 'log calculated weight recipient in kg')
#62516 missing >15%, logtransform
#difference with wgt_cm_tcr?

In [ ]:
%%R

#calculated BMI recipient
histsum(Heart$bmi_calc, 'calculated BMI recipient')
#62865 missing >15%

In [ ]:
%%R

#distance in miles between donor hospital and transplantation center
histsum(Heart$distance, 'distance in miles between donor hospital and transplantation center')
logdist <- log10(Heart$distance + 1)
histsum(logdist, 'log distance in miles between donor hospital and transplantation center')
#63441 missinng >15%, logtransform

In [ ]:
%%R

#ventilation support after listing
barsum(Heart$vent_support_after_list, 'ventilation support after listing')
#possible subdivision N = no, Y = yes,U = unknown
#62008 missing

In [ ]:
%%R

#proteinuria
barsum(Heart$protein_urine, 'proteinuria')
#possible subdivision N = no, Y = yes,U = unknown
#61964 missing

In [ ]:
%%R

#history of previous myocardial infarct
barsum(Heart$hist_mi, 'history of previous myocardial infarct')
#possible subdivision N = no, Y = yes,U = unknown
#62004 missing

In [ ]:
%%R

#method measuring left ventricle ejection?; unclear
barsum(Heart$lv_eject_meth, 'method measuring left ventricle ejection?; unclear')
#94531 missing

In [ ]:
%%R

#left ventricle ejection donor
histsum(Heart$lv_eject, 'left ventricle ejection donor')
#94524 missing >15%
#any value under 15% is unreasonably low for a donor heart

In [ ]:
%%R

#angiogram in donorheart; unclear
barsum(Heart$coronary_angio, 'angiogram in donorheart; unclear')
#62352 missing
#1 no angio, 2= normal angio, 3= abnormal angio

In [ ]:
%%R

#number of vessels with 50% stenosis in donor heart
barsum(Heart$vessels_50sten, 'number of vessels with 50% stenosis in donor heart')
#85 missing

In [ ]:
%%R

#biopsy on donor heart? unclear
barsum(Heart$biopsy_dgn, 'biopsy on donor heart? unclear')
#66362 missing
#retransplant

In [ ]:
%%R

#donor tattoos
barsum(Heart$tattoos, 'donor tattoos')
#possible subdivision N = no, Y = yes,U = unknown
#61964 missing

In [ ]:
%%R

#IgG for ebv infection in donor
barsum(Heart$ebv_igg_cad_don, 'IgG for ebv infection in donor')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, C/I/PD unclear
#111213 missing

In [ ]:
%%R

#cdc risk factors for blood bourne disease
barsum(Heart$cdc_risk_hiv_don, 'cdc risk factors for blood bourne disease')
#possible subdivision N =negative, ND = not done, P = positive, U = unknown, C/I/PD unclear
#106362 missing

In [ ]:
%%R

#Expanded Donor Including Brain Dead and DCD (per Kidney allocation definition)
barsum(Heart$ecd_donor, 'Expanded Donor Including Brain Dead and DCD (per Kidney allocation definition)')
#76664 missing

In [ ]:
%%R

#hematocrit
histsum(Heart$hematocrit_don, 'hematocrit')
#106662 missing > 15%

In [ ]:
%%R

#not in dictionary
barsum(Heart$retransplant, 'not in dictionary')
#possible: times retransplantation

In [ ]:
%%R

#Combined PanelReactiveAantibody from UNOS variables
histsum(Heart$newpra, 'Combined PanelReactiveAantibody from UNOS variables')
logpra <- log10(Heart$newpra + 1)
histsum(logpra, 'log Combined PanelReactiveAantibody from UNOS variables')
#correct normalization?
#70063 missing

In [ ]:
%%R

#education
barsum(Heart$education, 'education')
#1(GRADE SCHOOL (0-8)) 3(HIGH SCHOOL (9-12) or GED) 4 (ATTENDED COLLEGE/TECHNICAL SCHOOL), 5 (ASSOCIATE/BACHELOR DEGREE), 6 (POST-COLLEGE GRADUATE DEGREE), 996 (N/A), 998 (missing)
#27402 missing

In [ ]:
%%R

#patient died
barsum(Heart$death, 'patient died')
#61949 missing
#confirmation needed; difference wikth deathr

In [ ]:
%%R

#graft status
barsum(Heart$deathr, 'graft status')
#61949 missing

In [ ]:
%%R

#follow up time in days
histsum(Heart$futd, 'follow up time in days')
logfutd <- log10(Heart$futd + 1)
histsum(logfutd, 'log follow up time in days')
#61949 missing >15%, logtransform

In [ ]:
%%R

#follow up time in years
histsum(Heart$futdy, 'follow up time in years')
logfutdy <- log10(Heart$futdy + 1)
histsum(logfutdy, 'log follow up time in years')
#61949 missing >15%, logtransform

In [ ]:
%%R

#height ratio
histsum(Heart$Height.ratio, 'height ratio')

In [ ]:
%%R

#weight ratio
histsum(Heart$weight.ratio, 'weight ratio')

In [ ]:
%%R


#congenital heart disease diagnosis
barsum(Heart$congenital, 'congenital heart disease diagnosis')

In [ ]:
%%R

#cmassratio; unclear
histsum(Heart$cmassratio, 'cmassratio; unclear')
logcmassr <- log10(Heart$cmassratio)
histsum(logcmassr, 'log cmassratio; unclear')
#75052 missing >15%, logtransform